In [3]:
YOUTUBE_API_KEY = ''#chave removida antes de colocar no github
YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'
TERMO_BUSCA = 'inteligencia artificial' #termo da consulta
NUMERO_VIDEOS = 40
DB = "DB3"

In [4]:
from googleapiclient.discovery import build
import json

In [5]:
youtube = build(YOUTUBE_API_SERVICE_NAME,YOUTUBE_API_VERSION, developerKey=YOUTUBE_API_KEY)

In [6]:
resultado_busca_video = youtube.search().list(
    q=TERMO_BUSCA,
    part='id,snippet',
    maxResults=NUMERO_VIDEOS,
    type = 'video',
    regionCode = 'BR'
  ).execute()

In [7]:
def salvaArquivo(numVideo,NumPagComent,arqJson):
  arquivo = open('ArquivosJsonColetadosYoutube/Extraido_' + DB +'/video_' + str(numVideo) + '_pg_' + str(NumPagComent) + '.json','wt')
  json.dump(arqJson,arquivo)
  arquivo.close()

In [8]:
arquivo = open('ArquivosJsonColetadosYoutube/Extraido_' + DB +'/Lista_videos.json','wt')
json.dump(resultado_busca_video,arquivo)
arquivo.close()

In [9]:

videoIdList = []
for video in resultado_busca_video.get('items'):
  videoIdList.append([video.get('id').get('videoId'), video.get('id').get('kind')])

In [10]:
videoCount = 0
for video in videoIdList:
  videoCount = videoCount + 1
  resultado_busca_comentarios = youtube.commentThreads().list(
          part="snippet,replies",
          maxResults=100,
          textFormat="plainText",
          videoId=video[0]
      ).execute()
      
  t = True
  pg = 1
  salvaArquivo(videoCount,pg,resultado_busca_comentarios)
  while t:
    if resultado_busca_comentarios.get('nextPageToken') :
      pg = pg+1
      resultado_busca_comentarios = youtube.commentThreads().list(
          part="snippet,replies",
          maxResults=100,
          pageToken=resultado_busca_comentarios.get('nextPageToken'),
          textFormat="plainText",
          videoId=video[0]
      ).execute()
      salvaArquivo(videoCount,pg,resultado_busca_comentarios)
      
    else:
      t = False


# Nova seção